## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

Таблица с данными сосотоит из 12 столбцов и 21525 строк. Типы данных: int(целые числа), float(число с плавающей точкой), object(строковые данные или разный тип). В двух столбцах **[days_employed], [total_income]** таблицы есть пропущенные значения.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
data[data['days_employed'].isnull()].count() 
#методом isnull() и count() узнаем сколько пропущено значений в столбцах days_employed и total_income 

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64

### Вывод

В столбцах `[days_employed]` и `[total_income]` пропущено 2174 значения из 21525. Значения *days_employed* - трудовой стаж в днях и *total_income* - доход в месяц, являются количественными переменными.
Вероятной причиной появления пропусков является изначальное невнесение данных оператором банка при заполнении анкеты клиента. 

In [4]:
# Вычислим срезднее значение по столбцам days_employed и total_income
# В связи с присутствием отрицательных значений в столбцах, возьмем модуль значений этих ячеек 
data['days_employed'] = data['days_employed'].apply(abs)
data['total_income'] = data['total_income'].apply(abs)

In [5]:
income_type_count = data.groupby(['income_type'])

In [6]:
income_type_count.mean()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
income_type,,,,,,,
безработный,0.500000,366413.652744,38.000000,0.500000,0.500000,0.500000,131339.751676
в декрете,2.000000,3296.759962,39.000000,1.000000,0.000000,1.000000,53829.130729
госслужащий,0.622344,3399.896902,40.636737,0.671693,0.857437,0.058944,170898.309923
компаньон,0.598623,2111.524398,39.697542,0.715438,1.032448,0.073943,202417.461462
пенсионер,0.132002,365003.491245,59.063019,0.913900,0.985737,0.056017,137127.465690
предприниматель,0.000000,520.848083,42.500000,0.000000,0.500000,0.000000,499163.144947
сотрудник,0.641784,2326.499216,39.821027,0.849627,0.955661,0.095422,161380.260488
студент,0.000000,578.751554,22.000000,0.000000,4.000000,0.000000,98201.625314


In [7]:
data['total_income'] = income_type_count['total_income'].apply(lambda i: i.fillna(i.mean())).astype('int')

In [8]:
data['days_employed'] = income_type_count['days_employed'].apply(lambda i: i.fillna(i.mean())).astype('int')

In [9]:
# вычислим среднее по столбцам
data_mean = data.groupby('income_type')['days_employed', 'total_income'].agg(['mean']).astype('int')
print(data_mean)

                days_employed total_income
                         mean         mean
income_type                               
безработный            366413       131339
в декрете                3296        53829
госслужащий              3399       170897
компаньон                2111       202416
пенсионер              365002       137126
предприниматель           520       499163
сотрудник                2325       161379
студент                   578        98201


In [10]:
#Проверим отсутсвие пустых ячеек
data[data['days_employed'].isnull()].count()  


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [11]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Вывод

Так как отсутвующие значения (трудового стажа и доходов в месяц) относятся к количественному типу, допускается их замена значениями *среднего арифметическиого* и *медианой* по выборке.  
Учитывая что в данных был мусор в виде отрицательных значений, для верного расчета среднего арифметического потребовалось применения функции модуля.  

Методом *fillna()* заполнены пропуски, согласно сгруппированным по виду занятости средним значениям в столбцах days_employed и total_income.

### Замена типа данных

In [12]:
#выведем 5 первых значений таблицы
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


Как можно заметить, в столбцах со строковыми данными, значения имеют разные символы - заглавные и строчные. С точки зрения python одинаковые значения с разным регистром - ***различны***.

In [13]:
# убедимся в этом вызвав метод unique() по столбцу education
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [14]:
# Приведем все значения в столбцах со строковыми данными в нижний регистр
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['income_type'] = data['income_type'].str.lower()
data['purpose'] = data['purpose'].str.lower()

#### Проверим данные во всех столбцах:

In [15]:
# выведем уникальные значения по столбцу children
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце `[children]` есть отрицательное значение, это противоречит сути значений столбца. Используем функцию модуля для изменения значения.

In [16]:
data['children'] = data['children'].apply(abs)

Выводя уникальные значения для остальных столбцов, обнаружим что мусор присутвует в столбце `[gender]`.

In [17]:
# Посчитаем количество строк с уникальными значениями
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

При анализе данных всей строки со значением **XNA** в столбце `[gender]` нельзя точно определить, какое значение пропущено. Принимая во внимание, что таких значений в таблице всего одно, мы можем удалить эту строку из DataFrame. Удаление одной строки не повлияет существенно на нашу выборку и на проведение исследования в целом.


Проведем замену вещественных типов данных *(float)* на цельночисленный *(int)* для более ясного отображения DataFrame:

In [18]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
#data['dob_years'] = data['dob_years'].astype('int')

### Вывод

При анализе данных в таблице обнаружили следующие виды ошибок и мусора:  
1. Разный регистр в одинаковых значениях
2. Отрицательные значения, там где их по смыслу данных быть не должно
3. Ошибочно введеные/отсутствующие данные

Избавление от мусора и изменение типов данных улучшает качество исследования, читаемость и удобство работы с данными.

### Обработка дубликатов

Дубликаты влияют на результаты исследований, поэтому необходимо очистить данные для дальнейшего исследования:

In [19]:
# Найдем количество дубликатов в таблице методом duplicated() и посчитаем их количество методом value_counts()
print('Количество дубликатов в таблице: {}'.format(data.duplicated().sum()))

Количество дубликатов в таблице: 71


In [20]:
# Удалим дубликаты значений методом duplicated() и проверим результат
data = data.drop_duplicates().reset_index(drop=True)
print('Количество дубликатов после применения метода "drop_duplicates": {}'.format(data.duplicated().sum()))

Количество дубликатов после применения метода "drop_duplicates": 0


### Вывод

Благодаря выше проделанной работе по замене типов данных, в таблице был найден и удален 71 дубликат. Основными причинами дубликатов являются ошибки при вводе данных и пропущенные значения. 

### Лемматизация

В столбце `[purpose]` записаны цели значения получения кредита. Определим наиболее встречающиеся слова в этих значениях.  
Для этого используем процесс **лемматизации** - приведение слова к его словарной форме.
Используем библиотеку _pymystem3_ разработанную сотрудниками Яндекс и функцию *Counter* для подсчета встречающихся значений.

In [21]:
from pymystem3 import Mystem
m = Mystem()
from collections import Counter
data['purpose_lemmas_column'] = data['purpose'].apply(m.lemmatize)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas_column
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"


In [22]:
purpose_unique = data['purpose'].unique()
purpose_lemmas = m.lemmatize(' '.join(purpose_unique))
Counter(purpose_lemmas)

Counter({'покупка': 10,
         ' ': 96,
         'жилье': 7,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 2,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})

In [23]:
df_purpose_lemmas = pd.DataFrame({'lemmas':['жилье', 'автомобиль',
                                'образование', 'свадьба', 'коммерческий'
                                , 'строительство', 'ремонт']}
                                )

In [24]:
df_purpose_lemmas

,lemmas
0,жилье
1,автомобиль
2,образование
3,свадьба
4,коммерческий
5,строительство
6,ремонт


### Вывод

Процесс лемматизации позволил выделить наиболее часто встречающиеся слова и упростить выделение групп целей получения кредита. Всего было выделено 7 основных групп `df_purpose_lemmas`. Индексы этих групп возможно в дальнейшем использовать для внесения  в таблицу с данными.

### Категоризация данных

#### Категоризируем данные по возрасту:  
1. возраст отсутствует
2. от 18 до 64 - взрослые
3. от 65 и выше - пенсионеры

In [25]:
def age_group(age):

        if age == 0:
            return 'Отсутствует'
        if 18 <= age <= 25:
            return 'Менее 25 лет'
        if age < 35:
            return 'От 25 до 35 лет'
        if age < 45:
            return 'От 35 до 45 лет'
        if age < 55:
            return 'От 45 до 55 лет'
        if age < 65:
            return 'От 55 до 65 лет'
        return 'От 65 и выше'

In [26]:
data['dob_years_group'] = data['dob_years'].apply(age_group)

In [27]:
data.groupby('dob_years_group').agg({'debt': ['count']})

,debt
,count
dob_years_group,
Менее 25 лет,1232
От 25 до 35 лет,4735
От 35 до 45 лет,5740
От 45 до 55 лет,4867
От 55 до 65 лет,3884
От 65 и выше,895
Отсутствует,101


#### Проведем категоризацию данных по столбцу children:
1. нет_детей,
2. есть_дети,
3. многодетные.

In [28]:
def amount_children(child):
        if child == 0:
            return 'нет_детей'
        if child <= 2:
            return 'есть_дети'
        return 'многодетные'

In [29]:
data['amount_children_group'] = data['children'].apply(amount_children)
data.groupby('amount_children_group').agg({'debt': ['count']})

,debt
,count
amount_children_group,
есть_дети,6907
многодетные,456
нет_детей,14091


#### Классифицируем данные по целям кредита

Для начала создадим новый столбец puropose_dict, в котором заменим близкие по смыслу знаечния колонки purpose. 

In [30]:
data['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [31]:
data['purpose_dict'] = data['purpose']
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas_column,dob_years_group,amount_children_group,purpose_dict
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",От 35 до 45 лет,есть_дети,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",От 35 до 45 лет,есть_дети,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",От 25 до 35 лет,нет_детей,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",От 25 до 35 лет,многодетные,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",От 45 до 55 лет,нет_детей,сыграть свадьбу


In [32]:
def key_purpose(row):
    purp_dict = m.lemmatize(row)
    try:
        if 'строительство' in purp_dict:
            return 'строительство'
        if 'коммерческий' in purp_dict:
            return 'коммерческий'
        if 'ремонт' in purp_dict:
            return 'ремонт'
        if 'свадбьа' in purp_dict:
            return 'свадьба'
        if 'образование' in purp_dict:
            return 'образование'
        if 'строительство' in purp_dict:
            return 'строительство'
        if 'автомобиль' in purp_dict:
            return 'автомобиль'
        else:
            return 'жилье'
    except:
        return 'ошибка'

In [33]:
data['purpose_dict'] = data['purpose_dict'].apply(key_purpose)

Посчитаем количество для каждой группы целей кредита:

In [34]:
data.groupby('purpose_dict')['debt'].agg(['count']).sort_values(by='count', ascending=False)

,count
purpose_dict,
жилье,9339
автомобиль,4306
образование,4013
строительство,1878
коммерческий,1311
ремонт,607


### Вывод

В этом разделе выполнили категоризацию категориальных (цели кредита) и количстевнных переменных (возраст, количество детей).  
Классификация по типу кредита позволила наиболее точно охарактеризовать и обобщить цели кредита.  
Классифакицция по возрастным группам и количеству детей объединила значения в категории, что в результате поможет упростить работу по анализу данных.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [35]:
'Среднее значение просрочки по кредиту по всем клиентам {:.3f}'.format(data['debt'].mean())

'Среднее значение просрочки по кредиту по всем клиентам 0.081'

In [36]:
data.groupby('amount_children_group')['debt'].agg(['count', 'mean']).sort_values(by='mean', ascending=True)

,count,mean
amount_children_group,,
нет_детей,14091,0.075438
многодетные,456,0.085526
есть_дети,6907,0.092515


In [37]:
data.pivot_table(index = ['children'], values = 'debt', aggfunc = ['mean', 'count'])

,mean,count
,debt,debt
children,,
0,0.075438,14091
1,0.091658,4855
2,0.094542,2052
3,0.081818,330
4,0.097561,41
5,0.000000,9
20,0.105263,76


### Вывод

Количество кредитов выдаваемых клиентам без детей лидирует.  
Доля просрочки у клиентов **с 1 или 2 детьми выше**, чем для остальных категорий.  
Вероятно это можно объяснить, тем что клиенты **без детей** испытывают меньшую финансовую нагрузку, а **многодетные семьи** подходят к вопросу получения кредита более осторожно и лучше оценивают свои финансовые возможности.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [38]:
data.groupby('family_status')['debt'].agg(['count', 'mean']).sort_values(by='mean', ascending=True)

,count,mean
family_status,,
вдовец / вдова,959,0.065693
в разводе,1195,0.071130
женат / замужем,12339,0.075452
гражданский брак,4151,0.093471
не женат / не замужем,2810,0.097509


### Вывод

Лидерам по кредитам являются клиенты с семейным положением:
- женат / замужем, 
- в разводе, 
- вдовец / вдова.  
Эти категории клиентов имеют хороший показатель отсутствия просрочек. 

Больше всего просрочек по кредиту у категории клиентов:
- не женат / не замужем,
- гражданский брак.  

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [39]:
data['total_income_group'] = pd.cut(data['total_income'], [0,50000,100000,150000,200000,250000, 300000, 3000000])

In [40]:
data.groupby('total_income_group')['debt'].agg(['count', 'mean'])

,count,mean
total_income_group,,
"(0, 50000]",372,0.061828
"(50000, 100000]",4091,0.080909
"(100000, 150000]",6090,0.086371
"(150000, 200000]",5333,0.088693
"(200000, 250000]",2755,0.070417
"(250000, 300000]",1330,0.066165
"(300000, 3000000]",1483,0.071477


### Вывод

Из анализа таблицы уровня доходов и возврата кредита можно заметить, что просрочка по кредитам растет с увеличением доходов и достигает своего пика при заработке в 150000-200000 и уменьшается при доходах свыше 200000.  


Это можно объяснить, что **клиентам с низкими доходами** сумма по одобряемым кредитам ниже, поэтому и вернуть такой кредит легче.  
Клиенты со **средними доходами** больше берут кредитов и чаще имеют просрочки.  
Клиентам с **высоким достатком** проще обслуживать кредит, поэтому и просрочек по ним меньше.

- Как разные цели кредита влияют на его возврат в срок?

In [41]:
data.groupby('purpose_dict')['debt'].agg(['count', 'mean']).sort_values(by='mean', ascending=False)

,count,mean
purpose_dict,,
автомобиль,4306,0.093590
образование,4013,0.092200
строительство,1878,0.076677
коммерческий,1311,0.075515
жилье,9339,0.073884
ремонт,607,0.057661


### Вывод

Наибольшее число выдаваемых кредитов относится к категории **жилье**.  

Топ категорий по просрочкам по целям кредита являются:
 - автомобиль,
 - образование.
 
В случае с автомобилем клиенты веротяно чаще недооценивают свое финансовое положение и покупают автомибль дороже,чем могут себе позволить.

В случае с кредитом на образование, просрочка выше из-за стоимости образования и отсутствия примлемых доходов в период обучения.

Лучшими показателми по отсутвию просрочек являются категории **связанные с недвижимостью**:
- строительство,
- коммерчикая недвижимость,
- жилье,
- ремонт.


### Шаг 4. Общий вывод

#### При анализе данных были выявлены следующие проблемы, требующие решения:
1. Разный регистр в одинаковых значениях
2. Отрицательные значения, ошибочно применяемые в столбце
3. Неверно введеные/отсутствующие данные
4. Большое количество значений целей кредита, имеющих одинаковый смысл
5. Отсутсвие категорий по переменным - количество детей и возраст клиента  

#### Рекомендации
- Необходимо создать программные запреты на внеснение кредитными специалистами данных противоречащих значениям переменной.
- Не принимать заявки на кредит без полностью заполненных всех ячеек значений
- Проанализировать и категоризировать цели получения кредита и предоставить выбор цели только из соответствующего списка.
- Дополнительно категоризирваоть клиентов по количеству детей и возрасту.


#### Подводя итог можно сказать, что клиенты с наименьшими просрочками являются:
- клиенты у которых ***нет детей***,
- с семейным положением:
    - ***женат / замужем***, 
    - ***в разводе***, 
    - ***вдовец / вдова***.
- клиенты с ***урочнем дохода более 200 000***,
- с целями кредитов связанных с ***недвижимостью***, как с коммерческой, так и с личной.


#### Внимательно проводить скоринг следующих типов клиентов:
- ***с детьми***,
- семейным положением:
    - ***не женат / не замужем***,
    - ***гражданский брак***.
- клиенты со средним ***уровнем дохода от 50 000 до 200 000***,
- с целями кредита направленных на:
    - ***автомобиль***,
    - ***образование***.